In [1]:
# Things to add
## another table so concerts can have multiple headliners (maybe go off of concert_id)
## table for venue_type 
## way to add entries

###### documentation ######

## Import Data

In [2]:
import pandas as pd
data = pd.read_csv('tConcert.csv')
venues = pd.read_csv('tVenue.csv')

#data = data.merge(venues, how = 'left', on = ['venue_name'])

/Users/meganchurch/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
#tRole = data[['role_name']].drop_duplicates().reset_index(drop=True)
#tRole

## Create Database

In [4]:
import sqlite3

conn = sqlite3.connect('concert_database') 
c = conn.cursor()
c.execute("PRAGMA foregin_keys=ON;")

In [5]:
c.execute("DROP TABLE IF EXISTS tGenre;")
c.execute('''CREATE TABLE IF NOT EXISTS tGenre(
             genre_id INTEGER PRIMARY KEY AUTOINCREMENT, 
             genre_name TEXT)
             ''')

c.execute("DROP TABLE IF EXISTS tArtist;")
c.execute('''CREATE TABLE IF NOT EXISTS tArtist(
             artist_id INTEGER PRIMARY KEY AUTOINCREMENT, 
             artist_name TEXT,
             genre_id INTEGER REFERENCES tGenre(genre_id))
             ''')

c.execute("DROP TABLE IF EXISTS tVenue;")
c.execute('''CREATE TABLE IF NOT EXISTS tVenue(
             venue_id INTEGER PRIMARY KEY AUTOINCREMENT, 
             venue_name TEXT, 
             city TEXT, 
             state TEXT, 
             country TEXT, 
             capacity INTEGER, 
             venue_type TEXT, 
             longitude INTEGER, 
             latitude INTEGER)
             ''')

c.execute("DROP TABLE IF EXISTS tConcert;")
c.execute('''CREATE TABLE IF NOT EXISTS tConcert(
             concert_id INTEGER PRIMARY KEY,
             concert_name TEXT, 
             artist_id INTEGER REFERENCES tArtist(artist_id),
             venue_id INTEGER REFERENCES tVenue(venue_id),
             date TEXT NOT NULL)
             ''')

c.execute("DROP TABLE IF EXISTS tRole;")
c.execute('''CREATE TABLE IF NOT EXISTS tRole(
             role_id INTEGER PRIMARY KEY AUTOINCREMENT, 
             role_name TEXT)
             ''')

c.execute("DROP TABLE IF EXISTS tConcertRole;")
c.execute('''CREATE TABLE IF NOT EXISTS tConcertRole(
             concert_id INTEGER REFERENCES tConcert(concert_id), 
             artist_id INTEGER REFERENCES tArtist(artist_id), 
             role_id INTEGER REFERENCES tRole(role_id),
             PRIMARY KEY (concert_id, artist_id, role_id))
             ''')

In [6]:
conn.commit()

## Fill Functions

In [7]:
def FillTable(TableName, data, c):
    '''Load data onto db table, assumes that columns in the database
       and the dataframe have the same name'''
    i=0
    sql = "INSERT INTO " + TableName + " (" + \
          ",".join([c for c in data.columns]) + \
          ") VALUES (" + ','.join([':' + c for c in data.columns]) + ");"
    for row in data.to_dict(orient='records'):
        try:
            c.execute(sql,row)
            i+=1
        except:
            print(row)
            print(i)
            break

In [8]:
def GetGenreID(genre_name, conn, c):
    '''Check to see if this genre exists, if not create a new record'''
    
    sql = """SELECT genre_id 
             FROM tGenre 
             WHERE genre_name=?;"""
    
    insert = """INSERT INTO tGenre (genre_name) 
                    VALUES(?);"""
    
    genre = pd.read_sql(sql,conn,params = (genre_name,))

    if len(genre) == 0: 
        c.execute(insert, (genre_name,))
        genre = pd.read_sql(sql,conn,params = (genre_name,))

    if len(genre) > 1: 
        print('multiple genres')
        return None 

    return genre['genre_id'][0]

In [9]:
def GetRoleID(role_name, conn, c):
    '''Check to see if this role exists, if not create a new record'''
    
    sql = """SELECT role_id 
             FROM tRole 
             WHERE role_name=?;"""
    
    insert = """INSERT INTO tRole (role_name) 
                    VALUES(?);"""
    
    role = pd.read_sql(sql,conn,params = (role_name,))

    if len(role) == 0: 
        c.execute(insert, (role_name,))
        role = pd.read_sql(sql,conn,params = (role_name,))

    if len(role) > 1: 
        print('multiple roles')
        return None 

    if len(role) > 0:
        return role['role_id'][0]
    
    else:
        print('No role record found')
        return None

In [10]:
def GetArtistID(artist_name, genre_id, conn, c):
    '''Check to see if this artist exists, if not create a new record'''
    
    sql = """SELECT artist_id 
             FROM tArtist 
             WHERE artist_name=?
                 AND genre_id=?;"""
    
    insert = """INSERT INTO tArtist (artist_name,genre_id) 
                    VALUES(?,?);"""
    
    artist = pd.read_sql(sql,conn,params = (artist_name,genre_id,))

    if len(artist) == 0: 
        c.execute(insert, (artist_name,genre_id,))
        artist = pd.read_sql(sql,conn,params = (artist_name,genre_id,))

    if len(artist) > 1: 
        print('multiple artists')
        return None 

    return artist['artist_id'][0]

In [11]:
def GetVenueID(venue_name, city, state, country, capacity, venue_type, longitude, latitude, conn, c):
    '''Check to see if this venue exists, if not create a new record'''
    
    sql = """SELECT venue_id 
             FROM tVenue 
             WHERE venue_name=?
                 AND city=?
                 AND state=?
                 AND country=?
                 AND capacity=?
                 AND venue_type=? 
                 AND longitude=?
                 AND latitude=?;"""
    
    insert = """INSERT INTO tVenue (venue_name,city,state,country,capacity,venue_type,longitude,latitude) 
                    VALUES(?,?,?,?,?,?,?,?);"""
    
    venue = pd.read_sql(sql,conn,params = (venue_name,city,state,country,capacity,venue_type,longitude,latitude,))

    if len(venue) == 0: 
        c.execute(insert, (venue_name,city,state,country,capacity,venue_type,longitude,latitude,))
        venue = pd.read_sql(sql,conn,params = (venue_name,city,state,country,capacity,venue_type,longitude,latitude,))

    if len(venue) > 1: 
        print('multiple venues')
        return None 

    if len(venue) > 0:
        return venue['venue_id'][0]
    
    else:
        print('No venue record found')
        return None

## Fill Database 

In [12]:
# tVenue #
tVenue = venues[['venue_name','city','state','country','capacity','venue_type',
               'longitude','latitude']].drop_duplicates().reset_index(drop=True)
venue_id = []
for row in tVenue.values:
    venue_id.append(GetVenueID(row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], conn, c))
tVenue['venue_id'] = venue_id
conn.commit()

#join only venue_id to the main df 
data = data.merge(tVenue[['venue_name','venue_id']], how = 'left', on = ['venue_name'])

In [13]:
# tGenre #
tGenre = data[['genre_name']].drop_duplicates().reset_index(drop=True)
genre_id = []
for row in tGenre.values:
    genre_id.append(GetGenreID(row[0], conn, c))
tGenre['genre_id'] = genre_id
conn.commit()

data = data.merge(tGenre, how = 'left', on = ['genre_name'])

In [14]:
# tArtist #
tArtist = data[['artist_name','genre_id']].drop_duplicates().reset_index(drop=True)
artist_id = []
for row in tArtist.values:
    artist_id.append(GetArtistID(row[0], row[1], conn, c))
tArtist['artist_id'] = artist_id
conn.commit()

data = data.merge(tArtist, how = 'left', on = ['artist_name', 'genre_id'])

In [15]:
# tConcert #
tConcert = data[(data["role_name"] == "Headliner") | (data["role_name"] == "Second Headliner")]
tConcert = tConcert[['concert_id', 'concert_name', 'artist_id', 'venue_id', 'date']]
FillTable('tConcert', tConcert, c)

In [16]:
# tRole #
tRole = data[['role_name']].drop_duplicates().reset_index(drop=True)
role_id = []
for row in tRole.values:
    role_id.append(GetRoleID(row[0], conn, c))
tRole['role_id'] = role_id
conn.commit()

data = data.merge(tRole, how = 'left', on = ['role_name'])

In [17]:
# tConcertRole #
tConcertRole = data[['concert_id', 'artist_id', 'role_id']]
FillTable('tConcertRole', tConcertRole, c)

In [18]:
conn.commit()

## Check

In [26]:
pd.read_sql("""SELECT * FROM tVenue;""", conn)

,venue_id,venue_name,city,state,country,capacity,venue_type,longitude,latitude
0,1,9:30 Club,Washington D.C.,none,USA,1200,Club,-77.0237,38.9179
1,2,Acrisure Stadium,Pittsburgh,PA,USA,68400,Stadium,-80.0158,40.4468
2,3,Capital One Arena,Washington D.C.,none,USA,20356,Arena,-77.0209,38.8982
3,4,Eagle Bank Arena,Fairfax,VA,USA,10000,Arena,-77.3097,38.8269
4,5,Elevation 27,Virginia Beach,VA,USA,600,Club,-76.0195,36.8493
5,6,Gröna Lund,Stockholm,none,Sweden,30000,Amusement Park,18.0964,59.3234
6,7,Jiffy Lube Live,Bristow,VA,USA,25262,Amphitheater,-77.5896,38.7859
7,8,Merriweather Post Pavilion,Columbia,MD,USA,19319,Amphitheater,-76.8627,39.2089
8,9,MetLife Stadium,East Rutherford,NJ,USA,82500,Stadium,-74.0745,40.8135
9,10,Pier Six Pavilion,Baltamore,MD,USA,4600,Concert Hall,-76.6044,39.2840


In [27]:
conn.close()